In [7]:
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72

In [8]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [10]:
# Função auxiliar para enviar mensagem ao agente e obter resposta
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
##################################################################
# --- Agente 1: Buscador de Locais Confiáveis de Peças --- #
##################################################################
def agente_buscador_locais(tipo_item, marcas_modelos_interesse):
    buscador_locais = Agent(
        name='agente_buscador_locais',
        model='gemini-2.0-flash',
        description='Agente que busca online por locais confiáveis para comprar peças de carro e moto.',
        tools=[google_search],
        instruction=f'''
        Você é um especialista em encontrar fornecedores confiáveis de peças de carro e moto online.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para identificar locais online
        (como sites de ferro velhos especializados, desmanches certificados, lojas online com boa reputação,
        fóruns de proprietários com recomendações, etc.) que vendem {tipo_item} das seguintes marcas e modelos:
        {marcas_modelos_interesse}.

        Ao buscar, priorize locais que demonstrem confiabilidade através de:
        - Avaliações positivas de clientes.
        - Certificações ou selos de qualidade.
        - Políticas de devolução claras e justas.
        - Transparência nas informações dos produtos.
        - Reputação em comunidades online.

        Liste no máximo 5 locais promissores, fornecendo o nome do local (se aplicável) e a URL do site ou página de contato.
        Para cada local listado, inclua um breve resumo de por que ele parece ser um local confiável.
        '''
    )
    entrada_buscador_locais = f'Tipo de item: {tipo_item}\nMarcas e modelos de interesse: {marcas_modelos_interesse}'
    locais_encontrados = call_agent(buscador_locais, entrada_buscador_locais)
    return locais_encontrados

In [23]:
######################################################################
# --- Agente 2: Avaliador de Confiabilidade de Fornecedores --- #
######################################################################
def agente_avaliador_confiabilidade(lista_de_locais):
    avaliador_confiabilidade = Agent(
        name='agente_avaliador_confiabilidade',
        model='gemini-2.0-flash',
        description='Agente que avalia a confiabilidade de locais encontrados para compra de peças.',
        tools=[google_search],
        instruction='''
        Você é um investigador especializado em avaliar a confiabilidade de fornecedores online de peças de carro e moto.
        Com base na lista de locais fornecida, sua tarefa é usar a ferramenta de busca do Google (google_search)
        para investigar a reputação e confiabilidade de cada um deles.

        Para cada local, procure por:
        - Avaliações de clientes em diferentes plataformas (Google Reviews, redes sociais, sites de avaliação).
        - Menciona em fóruns ou comunidades de proprietários de veículos.
        - Artigos de notícias ou reportagens sobre o local.
        - Políticas de devolução e garantia.
        - Informações de contato claras e acessíveis.
        - Presença e engajamento em redes sociais.

        Para cada local avaliado, forneça um resumo da sua avaliação de confiabilidade (alta, média, baixa)
        e os principais pontos que justificam essa avaliação.
        '''
    )
    entrada_avaliador = f'Lista de locais a avaliar: {lista_de_locais}'
    avaliacao_fornecedores = call_agent(avaliador_confiabilidade, entrada_avaliador)
    return avaliacao_fornecedores

In [29]:
#########################################################################
# --- Agente 3 (Opcional): Agente de Contato Inicial (Formulando Perguntas) --- #
#########################################################################
def agente_contato_inicial(local, item_interesse, perguntas_extras=None):
    agente_contato = Agent(
        name='agente_contato_inicial',
        model='gemini-2.0-flash',
        description='Agente que formula e envia um contato inicial com um fornecedor para obter mais informações.',
        instruction=f'''
        Você é um assistente de compras especializado em adquirir peças de carro e moto para um ferro velho.
        Com base no nome e possível descrição do local fornecido e no item de interesse "{item_interesse}",
        sua tarefa é formular até três perguntas relevantes para obter informações essenciais antes de prosseguir com uma possível compra.
        Priorize as seguintes perguntas se fornecidas: {perguntas_extras if perguntas_extras else 'Nenhuma pergunta extra fornecida.'}.

        As perguntas devem abordar aspectos como:
        - Disponibilidade da peça específica.
        - Condição da peça (nova, usada, recondicionada).
        - Preço da peça.
        - Política de garantia ou devolução.
        - Opções de envio e custos (se aplicável, considerando que estamos em Nova Friburgo, RJ).
        - Outras informações relevantes para a compra.

         Após formular as perguntas, crie uma breve mensagem de contato (como um e-mail ou mensagem de formulário de contato)
        endereçada ao "[Nome do Local]" (se disponível na informação do local, caso contrário, use um genérico como "Prezado(s)"),
        apresentando o interesse na peça "{item_interesse}" e listando as perguntas formuladas de forma clara e profissional.
        ''' # Instrução atualizada para incluir perguntas_extras
    )



    entrada_contato = f'Local: {local}\nItem de interesse: {item_interesse}'
    mensagem_contato = call_agent(agente_contato, entrada_contato)
    return mensagem_contato

In [ ]:
# --- Execução do Sistema ---
print("🤖 Iniciando o Sistema de Monitoramento de Locais Confiáveis 🤖")

tipo_item_busca = input("❓ Que tipo de peça você está procurando (ex: motores, transmissões, peças de moto em geral)? ")
marcas_modelos = input("❓ Quais marcas e modelos de veículos/peças te interessam (ex: VW Gol, Honda CG 150)? ")

if not tipo_item_busca or not marcas_modelos:
    print("Por favor, forneça o tipo de item e as marcas/modelos de interesse.")
else:
    print(f"\n🔍 Buscando locais confiáveis para '{tipo_item_busca}' das marcas/modelos '{marcas_modelos}'...")
    locais_encontrados = agente_buscador_locais(tipo_item_busca, marcas_modelos)
    print("\n--- Locais Encontrados (Agente Buscador) ---")
    display(to_markdown(locais_encontrados))
    print("--------------------------------------------------")

    print("\n🧐 Avaliando a confiabilidade dos locais encontrados...")
    avaliacao_confiabilidade = agente_avaliador_confiabilidade(locais_encontrados)
    print("\n--- Avaliação de Confiabilidade (Agente Avaliador) ---")
    display(to_markdown(avaliacao_confiabilidade))
    print("------------------------------------------------------")

    # Exemplo de como usar o agente de contato (opcional)
    if "http" in locais_encontrados: # Simples verificação se URLs foram encontradas
         primeiro_local = locais_encontrados.split('\n')[0] # Pega a primeira linha como exemplo
         nome_do_local = primeiro_local.split(': ')[0] if ': ' in primeiro_local else "Fornecedor" # Tenta extrair o nome do local
         print(f"\n📧 Simulação de contato inicial com o local: {nome_do_local}...")
         mensagem_inicial = agente_contato_inicial(nome_do_local, tipo_item_busca) # Não passamos as perguntas aqui
         print("\n--- Mensagem de Contato Inicial (Agente de Contato) ---")
         display(to_markdown(mensagem_inicial))
         print("---------------------------------------------------------")

print("\n🏁 Sistema de Monitoramento de Locais Confiáveis Concluído. 🏁")